**Motor imagery-based EEG signal processing**<br/>
Presented by: Reza Saadatyar  2022-2023 <br/>
E-mail: Reza.Saadatyar92@gmail.com  <br/>
[Link](https://github.com/RezaSaadatyar/Motor-imagery-based-EEG-signal-processing)


================================== Importing the required Libraries ===========================

In [24]:
import numpy as np
import pandas as pd
from scipy import io
import matplotlib.pyplot as plt

========================================= Functions ==================================

In [21]:
# =============================================== Preparing data =========================================================
def Prepare_data(data, channels=None):  
    # ?------------------------------------------- Check type data -------------------------------------------      
    if 'DataFrame' not in str(type(data)):      
        data = pd.DataFrame(data)
    # !--------------------------------------- Check dimensional data ----------------------------------------
    if data.ndim < 3:
        if data.shape[0] < data.shape[1]:
            data = data.T
    elif data.ndim > 3:
        print("Please configure the data as two-dimensional (Sample * Channels)")
        sys.exit()
    # *--------------------------------------------- Set channels --------------------------------------------
    if channels is not None:
        data.columns=channels 
    else:
        data.columns = np.arange(1,data.shape[1]+1)
         
    return data, data.columns

*1. Importing the raw data*<br/>
[Dataset](https://www.bbci.de/competition/iv/desc_1.html)<br/>
*Parameters of data:*
  - *cnt:* the continuous EEG signals, size [time x channels]. The array is stored in datatype INT16. To convert it to uV values, use cnt= 0.1*double(cnt); in Matlab.
  - *mrk:* structure of target cue information with fields (the file of evaluation data does not contain this variable).
  - *pos:* vector of positions of the cue in the EEG signals given in unit sample, length #cues.
  - *y:* vector of target classes (-1 for class one or 1 for class two), length #cues.
  - *nfo:* structure providing additional information with fields
    - *fs:* sampling rate
    - *clab:* cell array of channel labels
    - *classes:* cell array of the names of the motor imagery classes
    - *xpos:* x-position of electrodes in a 2d-projection
    - *ypos:* y-position of electrodes in a 2d-projection

In [20]:
data_set = io.loadmat('BCICIV_calib_ds1a_100Hz.mat')
print(f"Variables = {list(data_set.keys())[3:]}")

Variables = ['cnt', 'mrk', 'nfo']


In [25]:
data = np.double(data_set['cnt']) * 0.1


In [ ]:
Nfo = data_set['nfo']
Nfo

In [ ]:
data, channels = Prepare_data(data, channels=None)

In [ ]:



Fs = 100
Time = np.arange(Data.shape[0]) / Fs

Mrk = data_set['mrk']
Time_Trial = np.array(Mrk['pos'].tolist()).flatten()